In [1]:
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import itertools
from itertools import chain, combinations

# The Evidence Soft Consistent Algorithm (ESC)
The ESC class implements a learning algorithm using multiple SGDClassifier experts.
It initializes masses for subsets of experts, with the initial mass set to 1 for the set of all experts. It makes aggregated predictions based on the experts' outputs. If the coarsened prediction results in an Abstention or Error,
the masses allocated to subsets of experts are updated according to the experts' prediction accuracy using the Shafer-Dempster rule (a refined mass of alpha is assigned to the set of experts who were correct, while those who were incorrect receive mass of 1-alpha).
When all experts agree, no update is made as the situation is uninformative.
The goal is to aggregate predictions in adversarial settings and minimize errors through dynamic mass updates.



In [2]:
class ESC:
    def __init__(self, num_experts, alpha, epsilon):
        self.total_abstensions = 0
        self.total_errors = 0
        self.num_experts = num_experts
        self.alpha = alpha
        self.epsilon = epsilon
        self.experts_info = {}  # Dictionary to store expert information
        self.experts = list(range(self.num_experts))

        # Dictionary to store masses allocated on the subsets of experts
        self.mass_function: dict[tuple[int, ...], float] = {
            tuple(sorted(subset)): 0.0
            for r in range(len(self.experts) + 1)
            for subset in itertools.combinations(sorted(self.experts), r)
        }

        # Set the mass of the full experts' set to 1.0
        self.mass_function[tuple(sorted(self.experts))] = 1.0

        print(f"Initialized power set and masses for {self.num_experts } experts: {self.mass_function}")

    def initialize_experts(self, X_train, y_train):
        """
        Initialize experts as SGDClassifier models.
        Each expert is trained on a subset of the training data.
        """

        for i in range(self.num_experts):
            X_subtrain, _, y_subtrain, _ = train_test_split(X_train, y_train, test_size=0.5)
            clf = SGDClassifier(max_iter=1000, tol=1e-3)
            clf.fit(X_subtrain, y_subtrain)

            # Store model information and empty predictions list for each expert
            self.experts_info[i] = {
                'model': clf,
                'predictions': []
            }

        print(f"Initialized {self.num_experts} experts: {self.experts_info}")

    def coarsening(self, t):
        """
        Aggregates the predictions of active experts and calculates the coarsened prediction.
        Returns the coarsened masses for predictions of 1, 0 or 01 in case of uncertainty.
        Returns the sets of experts predicting 1 and 0 at time t.
        """

        # Identify experts predicting 1 and 0 at time t
        experts_predicted_1 = {i for i, info in self.experts_info.items() if info['predictions'][t] == 1}
        experts_predicted_0 = {i for i, info in self.experts_info.items() if info['predictions'][t] == 0}

        print(f"Experts predicting 1: {experts_predicted_1}")
        print(f"Experts predicting 0: {experts_predicted_0}")

        mass_1, mass_0, mass_01 = 0, 0, 0

        # Compute the coarsened mass function
        for subset, mass in self.mass_function.items():
            if mass > 0:

                intersects_with_1 = experts_predicted_1.intersection(subset)
                intersects_with_0 = experts_predicted_0.intersection(subset)

                if intersects_with_1 and intersects_with_0:
                    print(f"Subset {subset} intersects with both: {intersects_with_1} and {intersects_with_0}")
                    mass_01 += mass
                elif intersects_with_1:
                    print(f"Subset {subset} intersects only with experts predicting 1: {intersects_with_1}")
                    mass_1 += mass
                elif intersects_with_0:
                    print(f"Subset {subset} intersects only with experts predicting 0: {intersects_with_0}")
                    mass_0 += mass

        print(f"Mass 1: {mass_1}, Mass 0: {mass_0}, Mass 01: {mass_01} ")

        return mass_0, mass_1, mass_01, experts_predicted_1, experts_predicted_0

    def update_masses(self, y_true, experts_predicted_1, experts_predicted_0):
        """
        Updates the experts' masses based on their predictions.
        Experts with zero mass are removed and the remaining masses are normalized if the total mass is grater than 1 + epsilon.
        Returns True if only one expert remains, otherwise returns False.
        """

        # Set the refined mass function for correct set (alpha) and incorrect set (1-alpha)
        refined_mass = {}
        if y_true == 1:
            refined_mass[frozenset(experts_predicted_1)] = self.alpha
            refined_mass[frozenset(experts_predicted_0)] = 1 - self.alpha
        else:
            refined_mass[frozenset(experts_predicted_1)] = 1 - self.alpha
            refined_mass[frozenset(experts_predicted_0)] = self.alpha

        for subset, mass in refined_mass.items():
            print(f"Refined subset: {subset}, Refined mass: {mass}")

        combined_mass = {}

        K = 0

        # Combine the current mass with the refined mass and compute the degree of conflict
        for subset, mass in self.mass_function.items():
            if mass > 0:
                print(f"\nConsidering subset {subset} with mass {mass}")
                for ref_subset, ref_mass in refined_mass.items():
                    intersection = frozenset(subset).intersection(ref_subset)
                    if intersection:
                        intersect_tuple = tuple(sorted(intersection))
                        print(f"Intersection between {subset} and {ref_subset}: {intersect_tuple}")
                        combined_mass[intersect_tuple] = combined_mass.get(intersect_tuple, 0) + (mass * ref_mass)
                        print(f"Updated combined mass for {intersect_tuple}: {combined_mass[intersect_tuple]}")
                    else:
                        print(f"No intersection between {subset} and {ref_subset}")
                        K += mass * ref_mass

        print(f"K conflict degree: {K}")

        # Normalization factor
        D = 1 - K

        print(f"1 - K: {D}")

        # Normalize the combined mass
        if K > 0:
            for subset in combined_mass:
                combined_mass[subset] /= D
                print(f"Normalized combined mass for {subset}: {combined_mass[subset]}")

        # Update the mass function with the new combined mass
        self.mass_function = {subset: combined_mass.get(tuple(sorted(subset)), 0) for subset in self.mass_function}
        print(f"Updated mass function: {self.mass_function}")

        # Calculate the total mass after the update
        total_mass = sum(self.mass_function.values())
        print(f"Total mass after update: {total_mass}")

        # Check if the total mass exceeds 1 + epsilon and normalize if necessary
        if total_mass > 1 + self.epsilon:
            print(f"Total mass exceeds 1 by {total_mass - 1}. Normalizing masses.")
            normalization_factor = 1 / total_mass
            for subset in self.mass_function:
                self.mass_function[subset] *= normalization_factor
            total_mass = sum(self.mass_function.values())
            print(f"Normalized total mass: {total_mass}")

        # Identify remaining experts
        remaining_experts = set()
        for subset, mass in self.mass_function.items():
            if mass > 0:
                remaining_experts.update(subset)

        print(f"Remaining experts: {remaining_experts}")

        # Update experts_info, keep just remaining experts
        self.experts_info = {expert: info for expert, info in self.experts_info.items() if expert in remaining_experts}
        print(f"Updated experts_info: {self.experts_info}")

        # Check if only one expert remains
        if len(remaining_experts) == 1:
            print(f"Only the perfect expert remains: {remaining_experts}")
            return True

        return False

    def run(self, X_test):
        """
        Runs the algorithm on the test data.
        """

        t = 0

        while t < len(X_test):

            print(f"\nRound: {t+1}")

            # Collect predictions from each expert
            for i in self.experts_info:
                prediction = self.experts_info[i]['model'].predict(X_test[t].reshape(1, -1))[0]
                self.experts_info[i]['predictions'].append(prediction)

            predictions_list = [self.experts_info[i]['predictions'][t] for i in self.experts_info]
            print(f"Experts predictions for round {t + 1}: {predictions_list}")

            # Check if all the predictions are the same
            all_equal = all(pred == predictions_list[0] for pred in predictions_list)

            # Generate the adversarial label
            y_true = 1 - (np.bincount(predictions_list).argmax())
            print(f"Adversarial true label for round {t + 1}: {y_true}")

            # Coarsening
            print("\nCoarsening:")
            mass_0, mass_1, mass_01, experts_predicted_1, experts_predicted_0 = self.coarsening(t)

            coarsened_pred = None

            # Determine the aggregated prediction
            if max(mass_0, mass_1, mass_01) == mass_0:
                coarsened_pred = 0
                print(f"Coarsened prediction for round {t + 1}: {coarsened_pred}")
            elif max(mass_0, mass_1, mass_01) == mass_1:
                coarsened_pred = 1
                print(f"Coarsened prediction for round {t + 1}: {coarsened_pred}")
            else:
                coarsened_pred = 'Abstention'
                print(f"Coarsened prediction for round {t + 1}: {coarsened_pred}")

            # If Abstention occurs, update the masses
            if coarsened_pred == 'Abstention':
                self.total_abstensions += 1
                print(f"Update total abstentions: {self.total_abstensions}")
                print("\nMass updating:")
                stop = self.update_masses(y_true, experts_predicted_1, experts_predicted_0)
                if stop:
                    break
            else:
                if coarsened_pred != y_true:
                    print("The aggregated prediction is different from the true label.")
                    self.total_errors += 1
                    print(f"Update total errors: {self.total_errors}")

                    # If experts agree completely, is not informative -> skip mass updating
                    if all_equal:
                        print("All experts agree completely. Skipping mass updating.")
                        t += 1
                        continue
                    else:
                      # If Errors occurs, update the masses
                      print("\nMass updating:")
                      stop = self.update_masses(y_true, experts_predicted_1, experts_predicted_0)
                      if stop:
                          break
                else:
                  print("The aggregated prediction is the same as the true label. No update is needed.")

            t += 1

        ########################################################################
        print()
        print("\nFinal Results:")
        print(f"\nTotal number of instances: {len(X_test)}")
        print(f"Remaining experts: {self.experts_info}")
        print(f"Total number of abstentions: {self.total_abstensions}")
        print(f"Total number of errors: {self.total_errors}")

In [3]:
# Test the algorithm
X, y = make_classification(n_samples=100, n_features=20, n_informative=15, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

num_experts = 3
alpha = 0.8
epsilon = 1e-10

esc = ESC(num_experts=num_experts, alpha = alpha, epsilon = epsilon)
esc.initialize_experts(X_train, y_train)
esc.run(X_test)

Initialized power set and masses for 3 experts: {(): 0.0, (0,): 0.0, (1,): 0.0, (2,): 0.0, (0, 1): 0.0, (0, 2): 0.0, (1, 2): 0.0, (0, 1, 2): 1.0}
Initialized 3 experts: {0: {'model': SGDClassifier(), 'predictions': []}, 1: {'model': SGDClassifier(), 'predictions': []}, 2: {'model': SGDClassifier(), 'predictions': []}}

Round: 1
Experts predictions for round 1: [1, 0, 1]
Adversarial true label for round 1: 0

Coarsening:
Experts predicting 1: {0, 2}
Experts predicting 0: {1}
Subset (0, 1, 2) intersects with both: {0, 2} and {1}
Mass 1: 0, Mass 0: 0, Mass 01: 1.0 
Coarsened prediction for round 1: Abstention
Update total abstentions: 1

Mass updating:
Refined subset: frozenset({0, 2}), Refined mass: 0.19999999999999996
Refined subset: frozenset({1}), Refined mass: 0.8

Considering subset (0, 1, 2) with mass 1.0
Intersection between (0, 1, 2) and frozenset({0, 2}): (0, 2)
Updated combined mass for (0, 2): 0.19999999999999996
Intersection between (0, 1, 2) and frozenset({1}): (1,)
Updated 